In [3]:
%load_ext autoreload
%autoreload 1

%run auxillary_functions.ipynb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import matplotlib.pyplot as plt
import numpy as np
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution